In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist

In [157]:
class inception(layers.Layer):
    def __init__(self, c1, c2, c3, c4):
        super().__init__()
        # 1*1卷积
        self.p1_1 = layers.Conv2D(c1, kernel_size=1, activation='relu', padding='same')
        # 1*1卷积+3*3卷积
        self.p2_1 = layers.Conv2D(c2[0], kernel_size=1, activation='relu', padding='same')
        self.p2_2 = layers.Conv2D(c2[1], kernel_size=3, activation='relu', padding='same')
        # 1*1卷积+5*5卷积
        self.p3_1 = layers.Conv2D(c3[0], kernel_size=1, activation='relu', padding='same')
        self.p3_2 = layers.Conv2D(c3[1], kernel_size=5, activation='relu', padding='same')
        # self.p3_2 = layers.Conv2D(c3[1], kernel_size=3, activation='relu', padding='same')
        # self.p3_3 = layers.Conv2D(c3[1], kernel_size=3, activation='relu', padding='same')
        # 3*3池化+1*1卷积
        self.p4_1 = layers.MaxPool2D(pool_size=3, padding='same',strides=1)
        self.p4_2 = layers.Conv2D(c4, kernel_size=1, padding='same', activation='relu')
    def call(self, x):
        # 线路1
        p1 = self.p1_1(x)
        # 线路2
        p2 = self.p2_2(self.p2_1(x))
        # 线路3
        p3 = self.p3_2(self.p3_1(x))
        # p3 = self.p3_3(self.p3_2(self.p3_1(x)))
        # 线路4
        p4 = self.p4_2(self.p4_1(x))
        # concat
        output = tf.concat([p1, p2, p3, p4], axis=-1)
        return output


In [159]:
# filter_size [卷积核个数, 全连接神经元数]
def aux_classifer(x, filter_size):
    # 平均池化
    x = layers.AveragePooling2D(pool_size=5, strides=3, padding='same')(x)
    # 1*1 卷积
    x = layers.Conv2D(filters=filter_size[0], kernel_size=1, strides=1, padding='valid', activation='relu')(x)
    x = layers.Flatten()(x)
    # 全连接层
    x = layers.Dense(units=filter_size[1], activation='relu')(x)
    #输出层
    x = layers.Dense(10, activation='softmax')(x)
    return x


## B1

In [162]:


# 输入
inputs = keras.Input(shape=(224, 224, 1), name='input')
#b1
# 7*7卷积
# x = layers.Conv2D(64, kernel_size=7, strides=2, padding='same', activation='relu')(inputs)
# 用两个3*3替换7*7. 识别效果提升
x = layers.Conv2D(32, kernel_size=3, strides=2, padding='same', activation='relu')(inputs)
x = layers.Conv2D(64, kernel_size=3, strides=1, padding='same', activation='relu')(x)
# # 3*3,s=2池化
x = layers.MaxPool2D(pool_size=3, strides=2, padding='same')(x)
x

<KerasTensor: shape=(None, 56, 56, 64) dtype=float32 (created by layer 'max_pooling2d_51')>

## B2

In [165]:

# b2
x = layers.Conv2D(64, kernel_size=1, padding='same', activation='relu')(x)
# 3*3 s=1 卷积 
x = layers.Conv2D(192, kernel_size=3, padding='same', activation='relu')(x)
# 3*3 s=2 池化 
x = layers.MaxPool2D(pool_size=3, strides=2, padding='same')(x)
x

<KerasTensor: shape=(None, 28, 28, 192) dtype=float32 (created by layer 'max_pooling2d_52')>

## B3

In [168]:

# 两个inception, 一个3*3 s=2 池化
x = inception(64, (96, 128), (16, 32), 32)(x)
x = inception(128, (128, 192), (32, 96), 64)(x)
x = layers.MaxPool2D(pool_size=3, strides=2, padding='same')(x)
x

<KerasTensor: shape=(None, 14, 14, 480) dtype=float32 (created by layer 'max_pooling2d_55')>

## B4

In [171]:
#inception
x = inception(192, (96, 208), (16, 48), 64)(x)
#辅助输出
aux_output_1 = aux_classifer(x, [128, 1024])
#inception
x = inception(160, (112, 224), (24, 64), 64)(x)
#inception
x = inception(128, (128, 256), (24, 64), 64)(x)
#inception
x = inception(112, (144, 288), (32, 64), 64)(x)
#辅助输出
aux_output_2 = aux_classifer(x, [128, 1024])
#inception
x = inception(256, (160, 320), (32, 128), 128)(x)
#池化
x = layers.MaxPool2D(pool_size=3, strides=2, padding='same')(x)
x

<KerasTensor: shape=(None, 7, 7, 832) dtype=float32 (created by layer 'max_pooling2d_61')>

## B5

In [174]:
x = inception(256, (160, 320), (32, 128), 128)(x)
x = inception(384, (192, 384), (48, 128), 128)(x)
#GAP
x = layers.GlobalAveragePooling2D()(x)
#输出
output = layers.Dense(10, activation='softmax', name='mainout')(x)
output

<KerasTensor: shape=(None, 10) dtype=float32 (created by layer 'mainout')>

In [176]:
model = tf.keras.Model(inputs=inputs, outputs=[output, aux_output_1, aux_output_2])

In [178]:
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input (InputLayer)          [(None, 224, 224, 1)]        0         []                            
                                                                                                  
 conv2d_222 (Conv2D)         (None, 112, 112, 32)         320       ['input[0][0]']               
                                                                                                  
 conv2d_223 (Conv2D)         (None, 112, 112, 64)         18496     ['conv2d_222[0][0]']          
                                                                                                  
 max_pooling2d_51 (MaxPooli  (None, 56, 56, 64)           0         ['conv2d_223[0][0]']          
 ng2D)                                                                                      

## 加载并处理数据集

In [181]:
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

In [183]:
import numpy as np
train_images = np.reshape(train_images, [train_images.shape[0],train_images.shape[1],train_images.shape[2],1])
test_images = np.reshape(test_images, [test_images.shape[0],test_images.shape[1],test_images.shape[2],1])
print(type(train_images))

<class 'numpy.ndarray'>


In [185]:
def get_train(size):
    index = np.random.randint(0, train_images.shape[0], size)
    resized_images = tf.image.resize_with_pad(train_images[index], 224, 224,)
    return resized_images.numpy(), train_labels[index]

def get_test(size):
    index = np.random.randint(0, test_images.shape[0], size)
    resized_images = tf.image.resize_with_pad(test_images[index], 224, 224,)
    return resized_images.numpy(), test_labels[index]


In [187]:
train_images, train_labels = get_train(256)
test_images, test_labels = get_test(128)

## 编译

In [218]:
optimizer = keras.optimizers.SGD(learning_rate=0.001, momentum=0.5)
callback = keras.callbacks.EarlyStopping(monitor='loss', patience=3)
model.compile(optimizer=optimizer,
             loss='sparse_categorical_crossentropy',
             metrics='accuracy',
             loss_weights=[1,0.3,0.3])

## 训练

In [223]:
model.fit(train_images, train_labels, epochs=10, verbose=1, validation_split=0.3, batch_size=10, callbacks=[callback])

Epoch 1/10
18/18 [==============================] - 6s 320ms/step - loss: 0.0019 - mainout_loss: 0.0019 - dense_14_loss: 2.1851e-05 - dense_16_loss: 1.9762e-05 - mainout_accuracy: 1.0000 - dense_14_accuracy: 1.0000 - dense_16_accuracy: 1.0000 - val_loss: 1.0307 - val_mainout_loss: 0.4829 - val_dense_14_loss: 0.7406 - val_dense_16_loss: 1.0855 - val_mainout_accuracy: 0.9221 - val_dense_14_accuracy: 0.8961 - val_dense_16_accuracy: 0.9091
Epoch 2/10
18/18 [==============================] - 6s 323ms/step - loss: 0.0018 - mainout_loss: 0.0018 - dense_14_loss: 2.1483e-05 - dense_16_loss: 1.9606e-05 - mainout_accuracy: 1.0000 - dense_14_accuracy: 1.0000 - dense_16_accuracy: 1.0000 - val_loss: 1.0354 - val_mainout_loss: 0.4844 - val_dense_14_loss: 0.7443 - val_dense_16_loss: 1.0925 - val_mainout_accuracy: 0.9221 - val_dense_14_accuracy: 0.8961 - val_dense_16_accuracy: 0.9091
Epoch 3/10
18/18 [==============================] - 6s 318ms/step - loss: 0.0017 - mainout_loss: 0.0017 - dense_14_loss:

In [225]:
model.evaluate(test_images, test_labels, verbose=1)

4/4 [==============================] - 1s 158ms/step - loss: 1.4024 - mainout_loss: 0.5830 - dense_14_loss: 1.4473 - dense_16_loss: 1.2841 - mainout_accuracy: 0.8828 - dense_14_accuracy: 0.8672 - dense_16_accuracy: 0.8906


[1.4024007320404053,
 0.5829648375511169,
 1.4473302364349365,
 1.2841224670410156,
 0.8828125,
 0.8671875,
 0.890625]